In [7]:
import pandas as pd
import numpy as np
import hopsworks
#import hsfs
from config_nb import Config

## 1. Data preparation

In [8]:
# We wanna see all columns of a DataFrame
pd.set_option("display.max_columns", None)

In [9]:
games = pd.read_csv('../jokic_murray/nuggets_2016_17_2022_23_r3.csv', \
                    dtype = {'SEASON_ID': 'string', 'GAME_ID': 'string'})
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PLAYOFFS,JOKIC_PTS,JOKIC_REB,JOKIC_AST,JOKIC_STARTER,MURRAY_PTS,MURRAY_REB,MURRAY_AST,MURRAY_STARTER,REST_PTS,REST_REB,REST_AST
0,22016,1610612743,DEN,Denver Nuggets,0021600010,2016-10-26,DEN @ NOP,W,240,107,37,83,0.446,8,24,0.333,25,33,0.758,11,47,58,18,7,4,24,22,5.0,0,5,7,2,1,1,4,1,0,101,47,15
1,22016,1610612743,DEN,Denver Nuggets,0021600033,2016-10-29,DEN vs. POR,L,265,113,38,100,0.380,6,23,0.261,31,39,0.795,16,45,61,19,10,5,17,21,-2.0,0,23,17,2,1,0,1,2,0,90,43,15
2,22016,1610612743,DEN,Denver Nuggets,0021600042,2016-10-31,DEN @ TOR,L,240,102,33,81,0.407,9,22,0.409,27,35,0.771,11,35,46,14,2,9,8,18,-3.0,0,12,4,1,1,1,2,2,0,89,40,11
3,22016,1610612743,DEN,Denver Nuggets,0021600067,2016-11-03,DEN @ MIN,W,240,102,39,81,0.481,6,22,0.273,18,25,0.720,7,37,44,24,10,2,20,24,3.0,0,8,3,5,1,0,3,4,1,94,38,15
4,22016,1610612743,DEN,Denver Nuggets,0021600082,2016-11-05,DEN @ DET,L,241,86,31,94,0.330,7,25,0.280,17,20,0.850,16,32,48,15,8,2,16,15,-17.0,0,6,6,0,1,9,1,2,1,71,41,13


In [10]:
games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PLAYOFFS,JOKIC_PTS,JOKIC_REB,JOKIC_AST,JOKIC_STARTER,MURRAY_PTS,MURRAY_REB,MURRAY_AST,MURRAY_STARTER,REST_PTS,REST_REB,REST_AST
618,42022,1610612743,DEN,Denver Nuggets,0042200401,2023-06-01,DEN vs. MIA,W,239,104,40,79,0.506,8,27,0.296,16,20,0.800,6,39,45,29,4,4,10,8,11.0,1,27,10,14,1,26,6,10,1,51,29,5
619,42022,1610612743,DEN,Denver Nuggets,0042200402,2023-06-04,DEN vs. MIA,L,239,108,39,75,0.520,11,28,0.393,19,22,0.864,9,29,38,23,7,2,13,21,-3.0,1,41,11,4,1,18,4,10,1,49,23,9
620,42022,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W,238,109,41,80,0.513,5,18,0.278,22,27,0.815,13,45,58,28,3,5,13,18,15.0,1,32,21,10,1,34,10,10,1,43,27,8
621,42022,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W,242,108,39,79,0.494,14,28,0.500,16,21,0.762,5,29,34,26,11,7,6,18,13.0,1,23,12,4,1,15,3,12,1,70,19,10
622,42022,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W,240,94,38,84,0.452,5,28,0.179,13,23,0.565,11,46,57,21,6,7,14,13,5.0,1,28,16,4,1,14,8,8,1,52,33,9


In [11]:
def final_prep(team_games: pd.DataFrame):
    
    #team_games['GAME_DATE'] = pd.to_datetime(team_games['GAME_DATE'])
    #team_games['TIMESTAMP'] = team_games['GAME_DATE'].values.astype(int) / (10**9)
    #team_games['TIMESTAMP'] = team_games['TIMESTAMP'].astype(int)
    team_games['WIN'] = np.where(team_games['WL'] == 'W', 1, 0)
    #team_games['GAME_ID'] = team_games['GAME_ID'].astype(int)
    
    cols = [col for col in team_games.columns if '_PTS' in col or '_REB' in col or '_AST' in col or '_STARTER' in col]
    cols.append('GAME_ID')
    cols.append('GAME_DATE')
    #cols.append('TIMESTAMP')
    cols.append('SEASON_ID')
    cols.append('PLAYOFFS')
    cols.append('WIN')
    team_games = team_games[cols].copy()
    
    for col in team_games.columns:
        team_games.rename(columns = {col: col.lower()}, inplace = True)
    
    return team_games

In [12]:
games = final_prep(games)
games.tail()

,jokic_pts,jokic_reb,jokic_ast,jokic_starter,murray_pts,murray_reb,murray_ast,murray_starter,rest_pts,rest_reb,rest_ast,game_id,game_date,season_id,playoffs,win
618,27,10,14,1,26,6,10,1,51,29,5,0042200401,2023-06-01,42022,1,1
619,41,11,4,1,18,4,10,1,49,23,9,0042200402,2023-06-04,42022,1,0
620,32,21,10,1,34,10,10,1,43,27,8,0042200403,2023-06-07,42022,1,1
621,23,12,4,1,15,3,12,1,70,19,10,0042200404,2023-06-09,42022,1,1
622,28,16,4,1,14,8,8,1,52,33,9,0042200405,2023-06-12,42022,1,1


In [13]:
games.shape

(623, 16)

In [14]:
games[(games['jokic_starter'] == 1) & (games['murray_starter'] == 1) & (games['season_id'].str[1:] <= '2018')].shape

(167, 16)

In [15]:
games[(games['jokic_starter'] == 1) & (games['murray_starter'] == 1) & (games['season_id'].str[1:] <= '2019')].shape

(245, 16)

A common rule-of-thumb indicates we need at least 10-20 observations per predictor to run a regression. As a conservative measure, we work with the upper bound. Thus, with 9 features (Jokic, Murray and rest of teammates points, rebounds and assists), we need at least 180 observations to run the analysis. With games from 2016-17 to 2019-20, we have more than 180 games where both Jokic and Murray were starters

In [16]:
games_2019 = games[games['season_id'].str[1:] <= '2019'].copy()
games_2019.tail()

,jokic_pts,jokic_reb,jokic_ast,jokic_starter,murray_pts,murray_reb,murray_ast,murray_starter,rest_pts,rest_reb,rest_ast,game_id,game_date,season_id,playoffs,win
347,21,6,2,1,21,1,5,1,72,30,16,0041900311,2020-09-18,42019,1,0
348,30,6,9,1,25,6,4,1,48,19,9,0041900312,2020-09-20,42019,1,0
349,22,10,5,1,28,8,12,1,64,26,9,0041900313,2020-09-22,42019,1,1
350,16,7,4,1,32,3,8,1,60,23,10,0041900314,2020-09-24,42019,1,0
351,20,7,5,1,19,4,8,1,68,25,10,0041900315,2020-09-26,42019,1,0


In [17]:
games_2019.shape

(352, 16)

## 2. Hopsworks

### 2.1. Basic info and connection

In [18]:
my_config = Config()
my_config.update_attributes_json()

In [19]:
project = hopsworks.login(
    project = my_config.HOPSWORKS_PROJECT_NAME,
    api_key_value = my_config.HOPSWORKS_API_KEY
)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/97430
Connected. Call `.close()` to terminate connection gracefully.


Alternative connection method. It needs the host name, which I don't know where to find :o. Working this way we can access a method to close the connection

```python
conn = hsfs.connection(
    project = my_config.HOPSWORKS_PROJECT_NAME,
    api_key_value = my_config.HOPSWORKS_API_KEY,
    #host = 
)
fs = conn.get_feature_store()
```

```python
conn.close()
```

### 2.2. Push data into feature store (create or get feature group)

Get or create feature group

In [20]:
fg = fs.get_or_create_feature_group(
    name = my_config.FEATURE_GROUP_NAME,
    version = 1,
    description = 'Games data from Denver Nuggets',
    primary_key = ['game_id'],
    online_enabled = True,
)

Push data to the feature group

In [21]:
fg.insert(
    games_2019,
    write_options = {'start_offline_backfill': False}
)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/97430/fs/97349/fg/111148


Uploading Dataframe: 0.00% |          | Rows 0/352 | Elapsed Time: 00:00 | Remaining Time: ?

(<hsfs.core.job.Job at 0x7fc0983bf9a0>, None)

```python
fg.insert(
    games_2019
)
```

```python
fg = fs.get_feature_group(
    name = my_config.FEATURE_GROUP_NAME,
    version = 1
)
```

```python
fg.delete()
```

```python
try:
    fg.delete()
except:
    print("Feature group doesn't exist")
```

### 2.3. Pull data from feature store (create or get feature view)

Get feature group

```python
fg = fs.get_feature_group(
    name = my_config.FEATURE_GROUP_NAME,
    version = 1
)
```

```python
try:
    fg = fs.get_feature_group(
        name = my_config.FEATURE_GROUP_NAME,
        version = 1
    )
except:
    print("Feature group doesn't exist")
```

```python
query = fg.select(['game_id'])
query.show(5)
```

Get feature view

In [22]:
fv = fs.get_or_create_feature_view(
    name = my_config.FEATURE_VIEW_NAME,
    version = 1,
    query = fg.select_all()
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/97430/fs/97349/fv/nuggets_games_view/version/1


Pull data from feature view

<span style="color:red">Allow some input to filter data, probably a date range</span>.

In [23]:
data_2019 = fv.get_feature_vectors(entry = [{'game_id': game_id} for game_id in games_2019['game_id'].to_list()])

Put data in a DataFrame

In [24]:
data_2019 = pd.DataFrame(data_2019, columns = games.columns)
data_2019.tail()

,jokic_pts,jokic_reb,jokic_ast,jokic_starter,murray_pts,murray_reb,murray_ast,murray_starter,rest_pts,rest_reb,rest_ast,game_id,game_date,season_id,playoffs,win
347,21,6,2,1,21,1,5,1,72,30,16,0041900311,2020-09-18,42019,1,0
348,30,6,9,1,25,6,4,1,48,19,9,0041900312,2020-09-20,42019,1,0
349,22,10,5,1,28,8,12,1,64,26,9,0041900313,2020-09-22,42019,1,1
350,16,7,4,1,32,3,8,1,60,23,10,0041900314,2020-09-24,42019,1,0
351,20,7,5,1,19,4,8,1,68,25,10,0041900315,2020-09-26,42019,1,0


In [25]:
data_2019.shape

(352, 16)

In [26]:
data_2019.dtypes

jokic_pts          int64
jokic_reb          int64
jokic_ast          int64
jokic_starter      int64
murray_pts         int64
murray_reb         int64
murray_ast         int64
murray_starter     int64
rest_pts           int64
rest_reb           int64
rest_ast           int64
game_id           object
game_date         object
season_id         object
playoffs           int64
win                int64
dtype: object

<span style="color:red">If the primary key is the date and the feature view uses the game id to retrieve the games, the dataframe will be empty, and viceversa</span>.

Regardless of the primary key ($game\_id$ or $game\_date$), I need a way to retrieve the games based on a range of dates: retrieve the games (from the `nba_api`) within this range and pull their ids or dates into a list. Retrieving the games will require to connect to the `nba_api`.

### 2.4. Push more data and retrieve it

Get data

In [27]:
games_2020 = games[games['season_id'].str[1:] == '2020'].copy()
games_2020.head()

,jokic_pts,jokic_reb,jokic_ast,jokic_starter,murray_pts,murray_reb,murray_ast,murray_starter,rest_pts,rest_reb,rest_ast,game_id,game_date,season_id,playoffs,win
352,29,15,14,1,9,4,1,1,84,27,13,0022000019,2020-12-23,22020,0,0
353,24,9,10,1,23,5,3,1,61,18,12,0022000009,2020-12-25,22020,0,0
354,19,12,18,1,21,5,2,1,84,35,11,0022000044,2020-12-28,22020,0,1
355,26,10,12,1,0,0,0,0,89,29,15,0022000055,2020-12-29,22020,0,0
356,17,9,11,1,31,4,5,1,55,25,12,0022000076,2021-01-01,22020,0,0


Push it

In [28]:
fg.insert(
    games_2020,
    write_options = {'start_offline_backfill': False}
)

Uploading Dataframe: 0.00% |          | Rows 0/82 | Elapsed Time: 00:00 | Remaining Time: ?

(<hsfs.core.job.Job at 0x7fc0983bf9a0>, None)

Retrieve it

In [29]:
fv = fs.get_feature_view(
    name = my_config.FEATURE_VIEW_NAME,
    version = 1,
)

In [30]:
games_list = games[games['season_id'].str[1:] <= '2020']['game_id'].to_list()
data_2016_2020 = fv.get_feature_vectors(entry = [{'game_id': game_id} for game_id in games_list])
data_2016_2020 = pd.DataFrame(data_2016_2020, columns = games.columns)
data_2016_2020.tail()

,jokic_pts,jokic_reb,jokic_ast,jokic_starter,murray_pts,murray_reb,murray_ast,murray_starter,rest_pts,rest_reb,rest_ast,game_id,game_date,season_id,playoffs,win
429,36,8,6,1,0,0,0,0,90,31,22,0042000166,2021-06-03,42020,1,1
430,22,9,3,1,0,0,0,0,83,32,23,0042000231,2021-06-07,42020,1,0
431,24,13,6,1,0,0,0,0,74,33,18,0042000232,2021-06-09,42020,1,0
432,32,20,10,1,0,0,0,0,70,26,11,0042000233,2021-06-11,42020,1,0
433,22,11,4,1,0,0,0,0,96,26,18,0042000234,2021-06-13,42020,1,0


In [31]:
data_2016_2020.shape

(434, 16)

In [32]:
data_2016_2020.dtypes

jokic_pts          int64
jokic_reb          int64
jokic_ast          int64
jokic_starter      int64
murray_pts         int64
murray_reb         int64
murray_ast         int64
murray_starter     int64
rest_pts           int64
rest_reb           int64
rest_ast           int64
game_id           object
game_date         object
season_id         object
playoffs           int64
win                int64
dtype: object

```python
fg.delete()
```